In [1]:

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import tensorflow
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers.core import Activation,Flatten,Dropout,Dense
from tensorflow.keras.optimizers import RMSprop ,SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
train_data_dir='/home/sahitya/Desktop/Projects/Age_gender_detection/fer2013/train'
validation_data_dir='/home/sahitya/Desktop/Projects/Age_gender_detection/fer2013/validation'

2022-09-19 14:00:57.675075: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-19 14:00:57.675125: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
validation_datagen=ImageDataGenerator(rescale=1./255)
validation_generator=validation_datagen.flow_from_directory(
train_data_dir,
color_mode='grayscale',
target_size=(48,48),
batch_size=500,
class_mode='categorical',
shuffle=True)

Found 28709 images belonging to 7 classes.


In [3]:
validation_generator=validation_datagen.flow_from_directory(
train_data_dir,
color_mode='grayscale',
target_size=(48,48),
batch_size=500,
class_mode='categorical',
shuffle=True)
class_labels=validation_generator.class_indices
class_labels={v:k for k,v in class_labels.items()}
classes=list(class_labels.values())
print(class_labels)

Found 28709 images belonging to 7 classes.
{0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}


In [4]:
import keras
classifier= keras.models.load_model("/home/sahitya/Desktop/saved_models/")


2022-09-19 14:01:07.834194: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-19 14:01:07.834235: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-19 14:01:07.834267: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sahitya-Inspiron-15-3567): /proc/driver/nvidia/version does not exist
2022-09-19 14:01:07.834544: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
from keras.models import load_model
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile,join
import re
import cv2
import numpy as np
from time import sleep
from keras.preprocessing.image import img_to_array

face_classifier=cv2.CascadeClassifier('/home/sahitya/Desktop/haarcascade_frontalface_default.xml')
def face_detector(img):
    
    
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    faces=face_classifier.detectMultiScale(gray,1.3,5)
    
    if faces is ():
        return (0,0,0,0),np.zeros((48,48),np.uint8),img
    
    
    for (x,y,w,h) in faces:

        cv2.rectangle(img,(x,y),(x+w,y+w),(255,0,0),2)
        
        roi_gray=gray[y:y+h,x:x+w]
        
    try:
        roi_gray=cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)
    except: 
        return (x,w,y,h),np.zeros((48,48),np.uint8),img
    return (x,w,y,h),roi_gray,img

    
    

<>:21: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:21: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_11503/1321735719.py:21: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


In [7]:
import cv2
cap=cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 3000)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 3000)

while True:
    ret,frame=cap.read()
    rect,face,image=face_detector(frame)
    
    if np.sum([face])!=0.0:
        
        roi=face.astype('float')/255.0
        roi=img_to_array(roi)
        
        roi=np.expand_dims(roi,axis=0)
        
        preds=classifier.predict(roi)[0]
        
        label=class_labels[preds.argmax()]
        
        label_position=(rect[0]+int((rect[1]/2)),rect[2]+25)
        cv2.putText(image,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    else:
        cv2.putText(image,'No Face Found',(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    
    cv2.imshow('All',image)
    if cv2.waitKey(1)==13:
        break
        
cap.release()
cv2.destroyAllWindows()

'/home/sahitya'